## Background  
This Jupyter Notebook calculates the percentage of households that **do not regularly use highways during peak-hour periods** by income groups.

Asana task: https://app.asana.com/1/11860278793487/project/12291104512646/task/1210119087413706?focus=true

## Inputs  
- `trip.csv`
- `household.csv`
- `trip_motorway_bridge_boolean.csv`: this is the output of the Jupyter Notebook `Generate_trip_motorway_booleans.ipynb`

## Outputs 
- `percent_noPNBMtrip_by_income.csv`: 

## Caveats 
- The results reported are representative of a typical Tue/Wed/Thu

- The `highway=motorway` tag in OpenStreetMap (OSM) is not always reliable. For this analysis, we have used it as-is and have not made corrections to its classification. It doesn't look too bad, and I'll share a map soon!

-  This analysis includes a time-of-day component. Specifically, it focuses on trips that use the freeway during the peak period. However, because the GeoDataFrame matched_path_gdf does not contain time stamps (double-check this!), I rely on the depart_hour and arrive_hour fields from the trip file. This introduces some limitations, since the freeway segment may occur at any point within the trip duration.

    For example:

      - If a trip starts at 9:45a and ends at 10:15a, and the first 15 minutes involved freeway travel, it is **correctly counted**.
      - If a trip starts at 9:45a and ends at 10:15a, but only the last 15 minutes involved freeway travel, it is **overcounted**.
      - If a trip starts at 5:45a and ends at 6:15a, but only the first 15 minutes involved freeway travel, it is **overcounted**.
      - If a trip starts at 5:45a and ends at 6:15a, and the last 15 minutes involved freeway travel, it is **correctly counted**.
  
## Still to do
- discuss defintion of "low income"
- visualize and review how good the highway=motorway tagging is, and how useful the bridge tagging is (this is done... notes to be added)


In [1]:
import os
import pandas as pd

# read the trip file
BATS_data_location = r"E:\Box\Modeling and Surveys\Surveys\Travel Diary Survey\BATS_2023\Data\2023\Full Weighted 2023 Dataset\WeightedDataset_02212025"
trip_df = pd.read_csv(os.path.join(BATS_data_location, "trip.csv"))

row_count_trip = len(trip_df)
print(f"Done reading the trip file. Number of rows: {row_count_trip}")


# read the household file
hh_df = pd.read_csv(os.path.join(BATS_data_location, "hh.csv"))

row_count_hh = len(hh_df)
print(f"Done reading the household file. Number of rows: {row_count_hh}")


Done reading the trip file. Number of rows: 365830
Done reading the household file. Number of rows: 8258


In [2]:
# Verify number of weighted households for each of Tue, Wed, Thu

# what is it in census? It should be about 2.8m. 
# The numbers below look low, because we do not have a household-day weight 

NumHh_Tue = hh_df.loc[hh_df["num_complete_tue"] == 1, "hh_weight_rmove_only"].sum()
print(f"Number of weighted households on Tue: {NumHh_Tue}")

NumHh_Wed = hh_df.loc[hh_df["num_complete_wed"] == 1, "hh_weight_rmove_only"].sum()
print(f"Number of weighted households on Wed: {NumHh_Wed}")

NumHh_Thu = hh_df.loc[hh_df["num_complete_thu"] == 1, "hh_weight_rmove_only"].sum()
print(f"Number of weighted households on Thu: {NumHh_Thu}")

Number of weighted households on Tue: 2151578.9136188542
Number of weighted households on Wed: 2160605.282457783
Number of weighted households on Thu: 2203531.932945707


In [3]:
# in order to have the right number of households to work with, we have to use data from Tue, Wed and Thu together
# the results will be representative of a typical Tue/Wed/Thu
NumHh = hh_df['hh_weight_rmove_only'].sum()
print(NumHh)

2884974.824949138


In [4]:
# read the file with the Has_motorway booleans
trip_motorway_bridge_df = pd.read_csv(os.path.join(BATS_data_location, "derived_variables", "trip_motorway_booleans.csv"))

# join the derived variables to the original trip file
trip_df = pd.merge(trip_df, trip_motorway_bridge_df, on="trip_id", how="left")

In [5]:
# check the number of rows (should be unchanged from before)
len(trip_df)

365830

In [6]:
# the geodata processing script produced two booleans.
# I'll focus on the one named has_nonBridge_motorway

trip_df["has_nonBridge_motorway"].value_counts(dropna=False)
# 1 means it is a trip that involves a freeway (that is not a bridge)
# 0 maens it is in the geodatabase as it is an auto trip, but didn't involve a freeway (that is not a bridge)
# NaN means it is not an auto trip as it wasn't in the geodatabase

# check the auto trips

NaN    158340
0.0    124345
1.0     83145
Name: has_nonBridge_motorway, dtype: int64

In [7]:
# tag trips that involve a freeway (that is not a bridge) and take place during peak hour
# AM Peak is 6 am to 10 am
# PM Peak is 3 pm to 7 pm
# Perferably, the peak hour or not tag should be done in the geodatabase. Unfortunately, it is not readily available


trip_df["is_Peak_nonBridge_motorway"] = (
    (trip_df["has_nonBridge_motorway"] == 1) &
    (
        ((trip_df["depart_hour"] >  6) & (trip_df["depart_hour"] < 10)) |
        ((trip_df["arrive_hour"] >  6) & (trip_df["arrive_hour"] < 10)) | 
        ((trip_df["depart_hour"] > 15) & (trip_df["depart_hour"] < 19)) |         
        ((trip_df["arrive_hour"] > 15) & (trip_df["arrive_hour"] < 19))
    ) 
)

# also tag trips that involve a freeway (that is not a bridge) but take place during any time of the day
trip_df["is_nonBridge_motorway"] = (trip_df["has_nonBridge_motorway"] == 1)

In [8]:
# for each household, sum the number of trips that involve a freeway (that is not a bridge) and take place during peak hour
# also do this for trips that involve a freeway (that is not a bridge) but take place during any time of the day
hh_peakNonBridgeMotorway_trips_df = trip_df.groupby("hh_id")[[
    "is_Peak_nonBridge_motorway",
    "is_nonBridge_motorway"    
]].sum().reset_index()

In [9]:
# Rename the columns so they have more intuitive names
hh_peakNonBridgeMotorway_trips_df = hh_peakNonBridgeMotorway_trips_df.rename(columns={
    "is_Peak_nonBridge_motorway": "numTrips_Peak_nonBridge_motorway",
    "is_nonBridge_motorway"     : "numTrips_nonBridge_motorway"
})

In [10]:
# join the info about peak non-bridge motorway trips to the household file

new_hh_df = pd.merge(
    hh_df,
    hh_peakNonBridgeMotorway_trips_df,
    on="hh_id",
    how="left" 
)

# check number of rows
row_count_newhh = len(new_hh_df)
print(f"Check number of rows in the new household file: {row_count_newhh}")

Check number of rows in the new household file: 8258


In [11]:
#look at the distribution of values in num_trip_peak_nonBridge_motorway
new_hh_df["numTrips_Peak_nonBridge_motorway"].value_counts(dropna=False)

# Notes on numTrips_Peak_nonBridge_motorway field:
# - NaN: The household didn't make any auto trips in the survey
# - 0: The household did not make any peak-period non-bridge motorway trips 
# - Other values: The number of peak-period non-bridge motorway trips made

# some households made many (more than 30) peak non-bridge motorway trips in the survey
# note that some households participated for 7 days, some participated for 1 day (or somewhere in between)

0.0     2540
1.0      933
2.0      761
3.0      571
4.0      489
5.0      398
6.0      353
7.0      268
8.0      252
9.0      217
NaN      192
10.0     185
11.0     154
12.0     139
13.0     109
14.0      88
16.0      73
15.0      72
17.0      58
18.0      49
19.0      43
20.0      37
22.0      35
23.0      34
21.0      28
24.0      21
26.0      20
25.0      19
27.0      16
29.0      10
34.0      10
28.0      10
35.0       9
31.0       6
30.0       6
33.0       6
32.0       6
39.0       5
36.0       4
40.0       4
48.0       3
37.0       3
52.0       2
43.0       2
41.0       2
49.0       2
59.0       1
71.0       1
54.0       1
46.0       1
51.0       1
60.0       1
67.0       1
47.0       1
55.0       1
56.0       1
58.0       1
45.0       1
69.0       1
42.0       1
Name: numTrips_Peak_nonBridge_motorway, dtype: int64

In [12]:
# Number of households that made no peak-period non-bridge motorway trip, weighted
numHh_MadeZero_Peak_nonBridge_motorway = new_hh_df.loc[
    new_hh_df["numTrips_Peak_nonBridge_motorway"] == 0,
    "hh_weight_rmove_only"
].sum()

print(f"The weighted number of hh that made no peak-period non-bridge motorway trip is: {numHh_MadeZero_Peak_nonBridge_motorway}")


The weighted number of hh that made no peak-period non-bridge motorway trip is: 511320.0907448505


In [13]:
# calculate % of households that do not use highways on a regular basis during peak hour periods
Percent_MadeZero_Peak_nonBridge_motorway = numHh_MadeZero_Peak_nonBridge_motorway / NumHh
print(f"The % of weighted households that made no peak-period non-bridge motorway trip is: {Percent_MadeZero_Peak_nonBridge_motorway}")


The % of weighted households that made no peak-period non-bridge motorway trip is: 0.17723554684878925


In [14]:
# Proceed with analysis by income
# Inspect the income variable
new_hh_df["income_broad"].value_counts(dropna=False)


5      2365
6      1952
4       878
3       790
999     778
1       756
2       739
Name: income_broad, dtype: int64

In [15]:
# Add income labels
income_labels = {
    1: "Under $25,000",
    2: "$25,000–$49,999",
    3: "$50,000–$74,999",
    4: "$75,000–$99,999",
    5: "$100,000–$199,999",
    6: "$200,000 or more",
    995: "Missing Response",
    999: "Prefer not to answer"
}

new_hh_df["income_broad_label"] = new_hh_df["income_broad"].map(income_labels)

In [16]:
new_hh_df["zero_Peak_nonBridge_motorway"] = new_hh_df["numTrips_Peak_nonBridge_motorway"] == 0
new_hh_df["zero_nonBridge_motorway"] = new_hh_df["numTrips_nonBridge_motorway"] == 0

new_hh_df["zero_Peak_nonBridge_motorway_weighted"] = new_hh_df["zero_Peak_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_nonBridge_motorway_weighted"]      = new_hh_df["zero_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]


# Group and aggregate
output_df = new_hh_df.groupby(["income_broad", "income_broad_label"], as_index=False).agg({
    "zero_Peak_nonBridge_motorway_weighted": "sum",
    "zero_nonBridge_motorway_weighted": "sum",
    "hh_weight_rmove_only": "sum"
})

# Add percent column
output_df["pct_zero_Peak_nonBridge_motorway_weighted"] = output_df["zero_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_nonBridge_motorway_weighted"]      = output_df["zero_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]

output_df

,income_broad,income_broad_label,zero_Peak_nonBridge_motorway_weighted,zero_nonBridge_motorway_weighted,hh_weight_rmove_only,pct_zero_Peak_nonBridge_motorway_weighted,pct_zero_nonBridge_motorway_weighted
0,1,"Under $25,000",119018.680665,97978.575280,273558.208651,0.435076,0.358164
1,2,"$25,000–$49,999",82951.456481,43871.454164,275291.333712,0.301322,0.159364
2,3,"$50,000–$74,999",61162.016114,30637.177956,301083.553489,0.203140,0.101756
3,4,"$75,000–$99,999",53033.162648,30232.826229,275829.625192,0.192268,0.109607
4,5,"$100,000–$199,999",87305.481589,41753.510049,657234.669681,0.132838,0.063529
5,6,"$200,000 or more",67600.748499,33420.124610,801675.590208,0.084324,0.041688
6,999,Prefer not to answer,40248.544748,19588.776779,300301.844016,0.134027,0.065230


## Related analysis -- share of hh that made a drive trip and transit trip

In [17]:
# Inspect the mode_type varaible
trip_df["mode_type"].value_counts(dropna=False)

8      217273
1      100866
13      20004
2        9217
995      8486
6        3878
7        1853
11       1425
14        892
3         630
9         376
12        338
5         281
4         262
10         49
Name: mode_type, dtype: int64

In [18]:
# For auto, filter trips to include only the following modes:
#    5. Taxi
#    6. TNC
#    8. Car
#    9. Carshare
#    11. Shuttle/vanpool
trip_df_AutoOnly    = trip_df[trip_df["mode_type"].isin([5, 6, 8, 9, 11])]

# For transit, filter trips to include only the following modes:
#    12. Ferry
#    13. Transit
trip_df_TransitOnly = trip_df[trip_df["mode_type"].isin([12, 13])]

In [19]:
# for each household, sum the number of auto trips  and transit trips
hh_Auto_trips_df    = trip_df_AutoOnly.groupby('hh_id').size().reset_index(name='numTrips_Auto')
hh_Transit_trips_df = trip_df_TransitOnly.groupby('hh_id').size().reset_index(name='numTrips_Transit')

In [20]:
list(hh_Auto_trips_df.columns)

['hh_id', 'numTrips_Auto']

In [21]:
list(hh_Transit_trips_df.columns)

['hh_id', 'numTrips_Transit']

In [22]:
len(hh_Auto_trips_df)

7401

In [23]:
len(hh_Transit_trips_df)

2829

In [24]:
# join the info about number of household auto trips to the household file

new_hh_df = pd.merge(
    new_hh_df,
    hh_Auto_trips_df,
    on="hh_id",
    how="left" 
)

# check number of rows
row_count_newhh = len(new_hh_df)
print(f"Check number of rows in the new household file: {row_count_newhh}")


Check number of rows in the new household file: 8258


In [25]:
# join the info about number of household transit trips to the household file

new_hh_df = pd.merge(
    new_hh_df,
    hh_Transit_trips_df,
    on="hh_id",
    how="left" 
)

# check number of rows
row_count_newhh = len(new_hh_df)
print(f"Check number of rows in the new household file: {row_count_newhh}")


Check number of rows in the new household file: 8258


In [33]:
#new_hh_df["zero_Peak_nonBridge_motorway"] = new_hh_df["numTrips_Peak_nonBridge_motorway"] == 0
new_hh_df["zero_auto"] = new_hh_df["numTrips_Auto"].isna()
new_hh_df["zero_transit"] = new_hh_df["numTrips_Transit"].isna()

#new_hh_df["zero_Peak_nonBridge_motorway_weighted"] = new_hh_df["zero_Peak_nonBridge_motorway"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_auto_weighted"] = new_hh_df["zero_auto"] * new_hh_df["hh_weight_rmove_only"]
new_hh_df["zero_transit_weighted"] = new_hh_df["zero_transit"] * new_hh_df["hh_weight_rmove_only"]

# Group and aggregate
output_df = new_hh_df.groupby(["income_broad", "income_broad_label"], as_index=False).agg({
    "zero_Peak_nonBridge_motorway_weighted": "sum",    
    "zero_nonBridge_motorway_weighted": "sum",   
    "zero_auto_weighted"   : "sum",
    "zero_transit_weighted": "sum",    
    "hh_weight_rmove_only" : "sum"
})

# Add percent column
output_df["pct_zero_Peak_nonBridge_motorway_weighted"] = output_df["zero_Peak_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_nonBridge_motorway_weighted"]      = output_df["zero_nonBridge_motorway_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_auto_weighted"] = output_df["zero_auto_weighted"] / output_df["hh_weight_rmove_only"]
output_df["pct_zero_transit_weighted"] = output_df["zero_transit_weighted"] / output_df["hh_weight_rmove_only"]

output_df

,income_broad,income_broad_label,zero_Peak_nonBridge_motorway_weighted,zero_nonBridge_motorway_weighted,zero_auto_weighted,zero_transit_weighted,hh_weight_rmove_only,pct_zero_Peak_nonBridge_motorway_weighted,pct_zero_nonBridge_motorway_weighted,pct_zero_auto_weighted,pct_zero_transit_weighted
0,1,"Under $25,000",119018.680665,97978.575280,63062.852794,182131.110130,273558.208651,0.435076,0.358164,0.230528,0.665786
1,2,"$25,000–$49,999",82951.456481,43871.454164,22081.794315,199313.402836,275291.333712,0.301322,0.159364,0.080212,0.724009
2,3,"$50,000–$74,999",61162.016114,30637.177956,10441.279420,232540.875948,301083.553489,0.203140,0.101756,0.034679,0.772347
3,4,"$75,000–$99,999",53033.162648,30232.826229,12182.926923,212460.013559,275829.625192,0.192268,0.109607,0.044168,0.770258
4,5,"$100,000–$199,999",87305.481589,41753.510049,11702.724752,502510.771572,657234.669681,0.132838,0.063529,0.017806,0.764583
5,6,"$200,000 or more",67600.748499,33420.124610,7788.360302,604203.668168,801675.590208,0.084324,0.041688,0.009715,0.753676
6,999,Prefer not to answer,40248.544748,19588.776779,17691.770522,228933.592211,300301.844016,0.134027,0.065230,0.058913,0.762345


In [34]:
# a simpler table showing just the percentages
output_pct_df = output_df[[
    "income_broad",
    "income_broad_label",    
    "pct_zero_Peak_nonBridge_motorway_weighted",
    "pct_zero_nonBridge_motorway_weighted",
    "pct_zero_auto_weighted",
    "pct_zero_transit_weighted"
]]

output_pct_df

,income_broad,income_broad_label,pct_zero_Peak_nonBridge_motorway_weighted,pct_zero_nonBridge_motorway_weighted,pct_zero_auto_weighted,pct_zero_transit_weighted
0,1,"Under $25,000",0.435076,0.358164,0.230528,0.665786
1,2,"$25,000–$49,999",0.301322,0.159364,0.080212,0.724009
2,3,"$50,000–$74,999",0.203140,0.101756,0.034679,0.772347
3,4,"$75,000–$99,999",0.192268,0.109607,0.044168,0.770258
4,5,"$100,000–$199,999",0.132838,0.063529,0.017806,0.764583
5,6,"$200,000 or more",0.084324,0.041688,0.009715,0.753676
6,999,Prefer not to answer,0.134027,0.065230,0.058913,0.762345
